# This is a note for [Camera Parameters - Extrinsics and Intrinsics (Cyrill Stachniss, 2020)](https://youtu.be/uHApDqH-8UE)

Personally, I prefer to setup the camera/image/sensor coordinate system as shown in the following [image](https://blender.stackexchange.com/questions/180391/how-to-get-z-distance-not-depth-from-camera-to-object-for-every-pixel-in-the-f). And I will use these coordinate system in this notebook.
![camera](./camera.png)

The one used in the video is a bit confusing because, there are two ways of forming the camera coordinate system in the video. One is at 6:20 and the image plane is behind the camera origin. The other one is at 23:19 and the z-axis is pointing away from the outside world. P.S. any of these two is good if you work out the math properly.